In [1]:
import numpy as np
import accumulator as Accum
from numba import njit


In [2]:
from importlib import reload

# Scalar data

In [3]:
reload(Accum)

<module 'accumulator' from '/Users/wpk/Documents/python/projects/python_dropin_files/accumulator.py'>

In [4]:
x = np.random.rand(1000)

In [5]:
s = Accum.StatsAccum.from_vals(x)

In [6]:
np.testing.assert_allclose(s.mean(), x.mean())
np.testing.assert_allclose(s.var(), x.var(ddof=0))

In [7]:
s = Accum.StatsAccum.from_stat(x.mean(), x.var(ddof=0), len(x))

In [8]:
sa = Accum.StatsArray.from_stats(x)

In [9]:
s

In [10]:
f = sa.to_xarray()

In [11]:
f

<xarray.DataArray (rec: 1000, val: 3)>
array([[1.        , 0.38241961, 0.        ],
       [1.        , 0.97699214, 0.        ],
       [1.        , 0.86117119, 0.        ],
       ...,
       [1.        , 0.35976153, 0.        ],
       [1.        , 0.6610002 , 0.        ],
       [1.        , 0.59220549, 0.        ]])
Coordinates:
  * val      (val) <U3 'cnt' 'ave' 'var'
Dimensions without coordinates: rec

In [12]:
import xarray as xr

In [13]:
xr.DataArray(s._data, dims='val', coords={'val':['cnt','ave','var']})

<xarray.DataArray (val: 3)>
array([1.00000000e+03, 4.91282729e-01, 8.53000249e-02])
Coordinates:
  * val      (val) <U3 'cnt' 'ave' 'var'

In [14]:
np.testing.assert_allclose(np.cumsum(x) / (np.arange(len(x)) + 1.0), sa.cummean())

In [15]:
sa.cumdata_last

array([1.00000000e+03, 4.91282729e-01, 8.53000249e-02])

In [16]:
np.prod(s.shape)

1.0

In [17]:
a = {'a':1} 
b = {}
d = {}
d.update(a)
d.update(b)
d

{'a': 1}

In [18]:
import xarray as xr

In [19]:
f = xr.DataArray(sa.data, dims=['rec','val'])

In [20]:
f.coords.update({'val':['a','b','c']})

In [21]:
f.coords['val'] = list('abc')

In [22]:
f

<xarray.DataArray (rec: 1000, val: 3)>
array([[1.        , 0.38241961, 0.        ],
       [1.        , 0.97699214, 0.        ],
       [1.        , 0.86117119, 0.        ],
       ...,
       [1.        , 0.35976153, 0.        ],
       [1.        , 0.6610002 , 0.        ],
       [1.        , 0.59220549, 0.        ]])
Coordinates:
  * val      (val) <U1 'a' 'b' 'c'
Dimensions without coordinates: rec

In [23]:
# addition

In [24]:
# x1 = np.random.rand(1000)
# x2 = np.random.rand(2000)
x1 = x[:500]
x2 = x[500:]

In [25]:
s1 = Accum.StatsAccum.from_vals(x1)
s2 = Accum.StatsAccum.from_vals(x2)

In [26]:
s12 = s1 + s2

In [27]:
s12._data - s._data

array([0.00000000e+00, 2.22044605e-16, 1.11022302e-16])

In [28]:
ss = s - s2
ss._data - s1._data

array([ 0.00000000e+00, -4.44089210e-16, -2.22044605e-16])

In [29]:
ss = s - s1
ss._data - s2._data

array([ 0.00000000e+00, -4.44089210e-16, -2.22044605e-16])

In [30]:
s = Accum.StatsAccum.from_stat(x.mean(), x.var(ddof=0), len(x))

In [31]:
np.testing.assert_allclose(s.mean(), x.mean())
np.testing.assert_allclose(s.var(), x.var(ddof=0))

# vector

In [32]:
reload(Accum)

<module 'accumulator' from '/Users/wpk/Documents/python/projects/python_dropin_files/accumulator.py'>

In [33]:
xs = np.random.rand(5000, 5)

xs1 = xs[:2500,...]
xs2 = xs[2500:,...]

x = xs.reshape(xs.shape[0], -1)

idx = np.cumsum(np.random.randint(1, 20, 1000))
idx = idx[idx < len(x)]
X = np.split(x, idx)
Xs = np.split(xs, idx, axis=0)

LA = [xx.mean(axis=0) for xx in X]
LV = [xx.var(axis=0, ddof=0) for xx in X]
LW = [xx.shape[0] for xx in X]

LA = np.array(LA)
LV = np.array(LV)
LW = np.array(LW)

LAs = [xx.mean(axis=0) for xx in Xs]
LVs = [xx.var(axis=0, ddof=0) for xx in Xs]
LWs = [xx.shape[0] for xx in Xs]

LAs = np.array(LAs)
LVs = np.array(LVs)
LWs = np.array(LWs)



In [34]:
s = Accum.StatsAccumVec.from_vals(xs)

In [35]:
w = Accum.StatsArray(shape=s.shape, child=Accum.StatsAccumVec)

In [36]:
w.push_stats(LA, LV, np.broadcast_to(LW[:,None], LA.shape))

In [37]:
LA.shape

(507, 5)

In [38]:
np.testing.assert_allclose(LA, w.mean())
np.testing.assert_allclose(LV, w.var())


In [39]:
s.mean()

array([0.5036704 , 0.49838795, 0.50379665, 0.50258301, 0.50365975])

In [40]:
w.cummean()[-1]

array([0.5036704 , 0.49838795, 0.50379665, 0.50258301, 0.50365975])

In [41]:
w.cumvar_last()

array([0.08316448, 0.08375163, 0.08252745, 0.08294445, 0.08403148])

In [42]:
w.cumstd_last()

array([0.28838252, 0.28939874, 0.28727591, 0.28800078, 0.28988184])

In [43]:
s.var()

array([0.08316448, 0.08375163, 0.08252745, 0.08294445, 0.08403148])

In [44]:
s.mean()

array([0.5036704 , 0.49838795, 0.50379665, 0.50258301, 0.50365975])

In [45]:
LAs.shape

(507, 5)

In [46]:
LWs.shape

(507,)

In [47]:
LAs.shape

(507, 5)

In [48]:
w = np.broadcast_to(LWs[:,None], LAs.shape)

In [49]:
f = Accum.StatsArray.from_stats(LAs, LVs, w, child=Accum.StatsAccumVec, shape=xs.shape[1:])

In [50]:
np.testing.assert_allclose(s.mean(), xs.mean(axis=0))
np.testing.assert_allclose(s.var(), xs.var(ddof=0, axis=0))
np.testing.assert_allclose(s.weight(), xs.shape[0])

In [51]:
s = Accum.StatsAccumVec.from_stat(xs.mean(axis=0), xs.var(axis=0, ddof=0), xs.shape[0])

In [52]:
np.testing.assert_allclose(s.mean(), xs.mean(axis=0))
np.testing.assert_allclose(s.var(), xs.var(ddof=0, axis=0))
np.testing.assert_allclose(s.weight(), xs.shape[0])

In [53]:
s = Accum.StatsAccumVec.from_stats(LAs, LVs, LWs[:, None])

In [54]:
np.testing.assert_allclose(s.mean(), xs.mean(axis=0))
np.testing.assert_allclose(s.var(), xs.var(ddof=0, axis=0))
np.testing.assert_allclose(s.weight(), xs.shape[0])

In [55]:
LAs.shape, LVs.shape, LWs.shape

((507, 5), (507, 5), (507,))

In [56]:
s = Accum.StatsAccumVec(shape=xs.shape[1:])
for a, v, w in zip(LAs, LVs, LWs):
    s.push_stat(a, v, w)

In [57]:
np.testing.assert_allclose(s.mean(), xs.mean(axis=0))
np.testing.assert_allclose(s.var(), xs.var(ddof=0, axis=0))
np.testing.assert_allclose(s.weight(), xs.shape[0])

In [58]:
s1 = Accum.StatsAccumVec.from_vals(xs1, axis=0)
s2 = Accum.StatsAccumVec.from_vals(xs2, axis=0)

In [59]:
s = s1 + s2

In [60]:
np.testing.assert_allclose(s.mean(), xs.mean(axis=0))
np.testing.assert_allclose(s.var(), xs.var(ddof=0, axis=0))
np.testing.assert_allclose(s.weight(), xs.shape[0])

In [61]:
ss = s - s1
np.testing.assert_allclose(ss.mean(), xs2.mean(axis=0))
np.testing.assert_allclose(ss.var(), xs2.var(ddof=0, axis=0))
np.testing.assert_allclose(ss.weight(), xs2.shape[0])

In [62]:
ss = s - s2
np.testing.assert_allclose(ss.mean(), xs1.mean(axis=0))
np.testing.assert_allclose(ss.var(), xs1.var(ddof=0, axis=0))
np.testing.assert_allclose(ss.weight(), xs1.shape[0])

## covariance unequal weight:

In [63]:
np.unravel_index(np.random.choice(np.arange(np.prod(x.shape)),50), x.shape)

(array([1766, 2903,  679, 2476, 3646, 4624, 2056,  491, 2649, 1216, 1075,
        1195,  928, 4702, 4162,  873, 2508, 3922, 3760, 1220, 4604, 4613,
        3423, 2560, 1171, 3083,  827, 4915, 2389, 1378,  481,  156, 4463,
         572, 1441, 2553, 3916,  206, 4729, 3084, 1323, 4113, 1892, 1000,
         880, 1026, 2394, 2351, 1237, 4522]),
 array([0, 4, 1, 2, 1, 0, 3, 2, 1, 3, 1, 3, 3, 3, 0, 2, 2, 1, 2, 0, 2, 2,
        4, 0, 4, 3, 2, 0, 0, 0, 4, 3, 0, 3, 1, 2, 1, 0, 4, 0, 0, 0, 4, 2,
        2, 2, 3, 3, 3, 0]))

In [64]:
x = np.random.rand(1000, 3,2)
xr = x.reshape(1000, -1)

n_nan = 100
idx = np.unravel_index(np.random.choice(np.arange(np.prod(x.shape)), size=50), x.shape)
x[idx] = np.nan

# a[0,1] = np.nan
# a[1,2] = np.nan
# a[2,3] = np.nan

# a[0,500+1]= np.nan
# a[1,500+2] = np.nan
# a[2,500+3] = np.nan

x1 = x[:500, ...]
x2 = x[500:, ...]

x1r = x1.reshape(x1.shape[0], -1)
x2r = x2.reshape(x2.shape[0], -1)


In [65]:
#%%timeit -n 3
ns = x.shape[1:] * 2
c = np.cov(xr.T, ddof=0).reshape(ns)
c1 = np.cov(x1r.T, ddof=0).reshape(ns)
c2 = np.cov(x2r.T, ddof=0).reshape(ns)

In [66]:
reload(Accum)

<module 'accumulator' from '/Users/wpk/Documents/python/projects/python_dropin_files/accumulator.py'>

In [67]:
s = Accum.StatsAccumCov(x.shape[1:])

In [68]:
for xx in x:
    s.push_val(xx)

In [69]:
np.testing.assert_allclose(s.mean_diag(), x.mean(axis=0))
np.testing.assert_allclose(s.var_diag(), x.var(ddof=0, axis=0))
np.testing.assert_allclose(s.var(), c)

In [70]:
s = Accum.StatsAccumCov(x.shape[1:])

s.push_vals(x, W=1.0)

np.testing.assert_allclose(s.mean_diag(), x.mean(axis=0))
np.testing.assert_allclose(s.var_diag(), x.var(ddof=0, axis=0))
np.testing.assert_allclose(s.var(), c)

In [71]:
s = Accum.StatsAccumCov.from_vals(x)

np.testing.assert_allclose(s.mean_diag(), x.mean(axis=0))
np.testing.assert_allclose(s.var_diag(), x.var(ddof=0, axis=0))
np.testing.assert_allclose(s.var(), c)

In [72]:
s1 = Accum.StatsAccumCov.from_vals(x1, W=1.0, axis=0)
s2 = Accum.StatsAccumCov.from_vals(x2, W=1.0, axis=0)

s = s1 + s2

np.testing.assert_allclose(s.mean_diag(), x.mean(axis=0))
np.testing.assert_allclose(s.var_diag(), x.var(ddof=0, axis=0))
np.testing.assert_allclose(s.var(), c)

s112 = s1 + s1 + s2

s = s112 - s1

np.testing.assert_allclose(s.mean_diag(), x.mean(axis=0))
np.testing.assert_allclose(s.var_diag(), x.var(ddof=0, axis=0))
np.testing.assert_allclose(s.var(), c)


In [73]:
s = Accum.StatsAccumCov(x.shape[1:])
s.push_stat(x.mean(axis=0), c, w=x.shape[0])
np.testing.assert_allclose(s.mean_diag(), x.mean(axis=0))
np.testing.assert_allclose(s.var_diag(), x.var(ddof=0, axis=0))
np.testing.assert_allclose(s.var(), c)


In [74]:
s = Accum.StatsAccumCov.from_stat(x.mean(axis=0), c, w=x.shape[0])
np.testing.assert_allclose(s.mean_diag(), x.mean(axis=0))
np.testing.assert_allclose(s.var_diag(), x.var(ddof=0, axis=0))
np.testing.assert_allclose(s.var(), c)


In [75]:
def get_cov(a, ns):

    ave = np.nanmean(a, axis=1)
    ave_ij = np.zeros((a.shape[0],)*2)

    cov = np.zeros((a.shape[0],)*2)
    cnt = np.zeros_like(cov)
    for var0 in range(a.shape[0]):
        for var1 in range(a.shape[0]):
            s = (a[var0, :] - ave[var0]) * (a[var1, :] - ave[var1])
            cnt[var0, var1] = np.isfinite(s).sum()
            cov[var0, var1] = np.nansum(s)#(a[var0, :] - ave[var0]) * (a[var1,:] - ave[var1]))
    if ns is None:
        ns = cnt.shape
    return (cov/cnt).reshape(ns), cnt.reshape(ns)
        
ns = x.shape[1:] * 2
a = xr.T
a1 = x1r.T
a2 = x2r.T
cov, cnt = get_cov(a,ns)
cov1, cnt1 = get_cov(a1,ns)
cov2, cnt2 = get_cov(a2,ns)



In [76]:
reload(Accum)

<module 'accumulator' from '/Users/wpk/Documents/python/projects/python_dropin_files/accumulator.py'>

In [77]:
s = Accum.StatsAccumCov(shape=x.shape[1:])

W = np.isfinite(x).astype(np.float)
W = W[...,None, None] * W[:, None, None, ...]

s.push_vals(x, W, axis=0)

np.testing.assert_allclose(s.mean_diag(), np.nanmean(x, axis=0))
np.testing.assert_allclose(s.weight(), cnt)
np.testing.assert_allclose(s.cov_corrected(),cov)

In [78]:
s = Accum.StatsAccumCov.from_vals(x, W)

In [79]:
np.testing.assert_allclose(s.mean_diag(), np.nanmean(x, axis=0))
np.testing.assert_allclose(s.weight(), cnt)
np.testing.assert_allclose(s.cov_corrected(),cov)
np.testing.assert_allclose(s.var_diag(), np.nanvar(x, ddof=0, axis=0))

In [80]:
Wi = []
for xx in [x1, x2]:
    w = np.isfinite(xx).astype(np.float)
    w = w[...,None, None] * w[:, None, None, ...]
    Wi.append(w)

In [81]:
s1 = Accum.StatsAccumCov.from_vals(x1, W=Wi[0])
s2 = Accum.StatsAccumCov.from_vals(x2, W=Wi[1])

In [82]:
s = s1 + s2

In [83]:
np.testing.assert_allclose(s.mean_diag(), np.nanmean(x, axis=0))
np.testing.assert_allclose(s.weight(), cnt)
np.testing.assert_allclose(s.cov_corrected(),cov)
np.testing.assert_allclose(s.var_diag(), np.nanvar(x, ddof=0, axis=0))

In [84]:
np.stack([1,2,3], axis=-1).shape

(3,)

In [85]:
W = np.array([s.weight(), s.weight()])
A = np.array([s.mean(), s.mean()])
V = np.array([s.var(), s.var()])

In [86]:
W.shape

(2, 3, 2, 3, 2)

In [87]:
np.stack([W, A, V], axis=-1).shape

(2, 3, 2, 3, 2, 3)

In [88]:
np.stack([s.weight(), s.mean(), s.var()], axis=-1)

array([[[[[ 9.95000000e+02,  4.87198080e-01,  7.84195861e-02],
          [ 9.85000000e+02,  4.86397687e-01,  2.93751787e-03]],

         [[ 9.91000000e+02,  4.87629875e-01,  6.14997532e-04],
          [ 9.83000000e+02,  4.87242595e-01, -2.34767649e-03]],

         [[ 9.89000000e+02,  4.86498854e-01, -1.01954396e-03],
          [ 9.82000000e+02,  4.88255170e-01, -1.01649386e-03]]],


        [[[ 9.85000000e+02,  5.06512410e-01,  2.93751787e-03],
          [ 9.90000000e+02,  5.06617039e-01,  8.43801609e-02]],

         [[ 9.86000000e+02,  5.06315144e-01, -5.79218616e-03],
          [ 9.78000000e+02,  5.05190189e-01,  4.41051733e-05]],

         [[ 9.84000000e+02,  5.06460322e-01, -4.67430630e-03],
          [ 9.77000000e+02,  5.06567381e-01,  2.75317105e-03]]]],



       [[[[ 9.91000000e+02,  4.83166734e-01,  6.14997532e-04],
          [ 9.86000000e+02,  4.83610670e-01, -5.79218616e-03]],

         [[ 9.96000000e+02,  4.83086364e-01,  7.91301627e-02],
          [ 9.84000000e+02,  4.8184

In [89]:
np.array([s.weight(), s.mean(), s.var()])

array([[[[[ 9.95000000e+02,  9.85000000e+02],
          [ 9.91000000e+02,  9.83000000e+02],
          [ 9.89000000e+02,  9.82000000e+02]],

         [[ 9.85000000e+02,  9.90000000e+02],
          [ 9.86000000e+02,  9.78000000e+02],
          [ 9.84000000e+02,  9.77000000e+02]]],


        [[[ 9.91000000e+02,  9.86000000e+02],
          [ 9.96000000e+02,  9.84000000e+02],
          [ 9.90000000e+02,  9.83000000e+02]],

         [[ 9.83000000e+02,  9.78000000e+02],
          [ 9.84000000e+02,  9.88000000e+02],
          [ 9.82000000e+02,  9.75000000e+02]]],


        [[[ 9.89000000e+02,  9.84000000e+02],
          [ 9.90000000e+02,  9.82000000e+02],
          [ 9.94000000e+02,  9.81000000e+02]],

         [[ 9.82000000e+02,  9.77000000e+02],
          [ 9.83000000e+02,  9.75000000e+02],
          [ 9.81000000e+02,  9.87000000e+02]]]],



       [[[[ 4.87198080e-01,  4.86397687e-01],
          [ 4.87629875e-01,  4.87242595e-01],
          [ 4.86498854e-01,  4.88255170e-01]],

         [[ 

In [96]:
@njit
def _push_val_cov_data_general(data, w, x):
    # data[i,j,0] = weight[i,j]
    # data[i,j,1] = ave[i,j]
    # data[i,j,2] = cov_prime[i,j]
    # cov[i,j] = cov_prime[i, j] + (ave[i,i] - ave[i,j]) * (ave[j,j] - ave[j,i])
    
    n = x.shape[0]
    for i in range(n):
        xi = x[i]
        if np.isnan(xi):
            continue
            
        # i = j
        weight = data[i, i, 0] + w
        alpha = w / weight
        delta = xi - data[i,i, 1]
        incr = delta * alpha
        
        data[i,i, 0]  = weight
        data[i,i, 1] += incr
        data[i,i, 2] = (1.0 - alpha) * (data[i, i, 2] + delta * incr)
        
            
        for j in range(i+1, n):
            xj = x[j]
            if np.isnan(xj): 
                continue
                
            weight = data[i, j, 0] + w
            alpha = w / weight
            avei = data[i,j, 1]
            deltai = xi - avei
            incri = deltai * alpha
            data[i, j, 0] = weight
            data[i, j, 1] = avei + incri
            
            avej = data[j, i, 1]
            deltaj = (xj - avej)
            incrj = deltaj * alpha
            data[j, i, 0] = weight
            data[j, i, 1] = avej + incrj
            
            # cov_prime
            v = (1.0 - alpha) * (data[i, j, 2] + incri * deltaj)
            
            data[i,j, 2] = v
            data[j,i, 2] = v
                    
            
            
@njit
def _push_val_cov(data, w, x):
    # data[i,j,0] = weight[i,j]
    # data[i,j,1] = ave[i,j]
    # data[i,j,2] = cov_prime[i,j]
    # cov[i,j] = cov_prime[i, j] + (ave[i,i] - ave[i,j]) * (ave[j,j] - ave[j,i])
    
    n = x.shape[0]
    for i in range(n):
        xi = x[i]
        for j in range(i, n):
            wij = w[i, j]
            if wij == 0:
                continue
            
            weight = data[i, j, 0] + wij
            alpha = wij / weight
            
            avei = data[i, j, 1]
            deltai = xi - avei
            incri = deltai * alpha
            data[i, j, 0] = weight
            data[i, j, 1] = avei + incri
            
            if j == i:
                # same
                data[i, j, 2] = (1.0 - alpha) * (data[i, j, 2] + deltai * incri)
            else:
                avej = data[j, i, 1]
                deltaj = x[j] - avej
                incrj = deltaj * alpha
                
                data[j, i, 0] = weight
                data[j, i, 1] = avej + incrj
                
                # cov_prime
                v = (1.0 - alpha) * (data[i, j, 2] + incri * deltaj)
            
                data[i, j, 2] = v
                data[j, i, 2] = v
            
            

                
    

In [97]:
#%%timeit -n 3 -r 3
# old way
n = a.shape[0]
data = np.zeros((n,2+n))
for xx in a.T:
    _push_val_cov_data(data, 1.0, xx)

NameError: name '_push_val_cov_data' is not defined

In [ ]:
#%%timeit -n 1 -r 1
n = a.shape[0]
data = np.zeros((n,n, 3))

for xx in a.T:
    _push_val_cov_data_general(data, 1.0, xx)

In [92]:
np.testing.assert_allclose(data[...,0], cnt)
np.testing.assert_allclose(data[...,1].diagonal(), np.nanmean(a, axis=-1))


# cov_xy = cov_prime_xy + (x_mean_x - x_mean_xy) * (y_mean_y - y_mean_xy)

mean = data[:, :, 1].diagonal()
cross = mean[:, None] - data[:, :, 1]
cross = cross * cross.T

c = data[:, :, 2] + cross

np.testing.assert_allclose(c, cov)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

(shapes (6,), (3, 2, 3, 2) mismatch)
 x: array([0., 0., 0., 0., 0., 0.])
 y: array([[[[995., 985.],
         [991., 983.],
         [989., 982.]],...

In [93]:
%%timeit -n 1 -r 1
# another way
n = a.shape[0]
data = np.zeros((n,n,3))
for xx in a.T:
    w = np.isfinite(xx).astype(np.float)
    w = w * w[:, None]
    _push_val_cov(data, w, xx)

229 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [94]:
#%%timeit -n 1 -r 1
# another way
n = a.shape[0]
data = np.zeros((n,n,3))
for xx in a.T:
    w = np.isfinite(xx).astype(np.float)
    w = w * w[:, None]
    _push_val_cov(data, w, xx)

In [95]:
np.testing.assert_allclose(data[...,0], cnt)
np.testing.assert_allclose(data[...,1].diagonal(), np.nanmean(a, axis=-1))


# cov_xy = cov_prime_xy + (x_mean_x - x_mean_xy) * (y_mean_y - y_mean_xy)

mean = data[:, :, 1].diagonal()
cross = mean[:, None] - data[:, :, 1]
cross = cross * cross.T

c = data[:, :, 2] + cross

np.testing.assert_allclose(c, cov)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

(shapes (6, 6), (3, 2, 3, 2) mismatch)
 x: array([[995., 985., 991., 983., 989., 982.],
       [985., 990., 986., 978., 984., 977.],
       [991., 986., 996., 984., 990., 983.],...
 y: array([[[[995., 985.],
         [991., 983.],
         [989., 982.]],...

In [36]:
(1,2,3)*2

(1, 2, 3, 1, 2, 3)

In [31]:
@njit
def _push_stat_cov(data, w, a, v):
    n = a.shape[0]
    for i in range(n):
        for j in range(i, n):
            wij = w[i, j]
            if wij == 0:
                continue
            
            
            weight = data[i, j, 0] + wij
            alpha = wij / weight
            
            avei = data[i, j, 1]
            deltai = a[i,j] - avei
            incri = deltai * alpha
            data[i, j, 0] = weight
            data[i, j, 1] = avei + incri
            
            if j == i:
                # same
                data[i, j, 2] = v[i,j] * alpha + (1.0 - alpha) * (data[i, j, 2] + deltai * incri)
            else:
                avej = data[j, i, 1]
                deltaj = a[j, i] - avej
                incrj = deltaj * alpha
                
                data[j, i, 0] = weight
                data[j, i, 1] = avej + incrj
                
                # cov_prime
                c = v[i, j] * alpha + (1.0 - alpha) * (data[i, j, 2] + incri * deltaj)
            
                data[i, j, 2] = c
                data[j, i, 2] = c
                
                
            
        

In [32]:
n = a.shape[0]
data1 = np.zeros((n,n,3))
data2 = np.zeros((n, n, 3))

for xx in a1.T:
    w = np.isfinite(xx).astype(np.float)
    w = w * w[:, None]
    _push_val_cov(data1, w, xx)
    
for xx in a2.T:
    w = np.isfinite(xx).astype(np.float)
    w = w * w[:, None]
    _push_val_cov(data2, w, xx)
    

In [35]:
data12 = np.zeros((n, n, 3))
_push_stat_cov(data12, data1[...,0], data1[...,1], data1[...,2])
_push_stat_cov(data12, data2[...,0], data2[...,1], data2[...,2])

In [34]:
np.testing.assert_allclose(data, data12)